In [1]:
import pandas as pd
from configurations import basic_parameters as bp

In [2]:
csv_dataframe = bp['dataframe_dir'] + '/' + bp['dataframe_file']

In [3]:
names = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

df = pd.read_csv(csv_dataframe, names=names, usecols=['Open time', 'Open', 'High', 'Low', 'Close'])

In [4]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [5]:
def stochastic(data, k_window, d_window, window):
    
    # input to function is one column from df
    # containing closing price or whatever value we want to extract K and D from
    
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()
    
    stoch = ( (data - min_val) / (max_val - min_val) ) * 100
    
    K = stoch.rolling(window=k_window, center=False).mean() 
    #K = stoch
    
    D = K.rolling(window=d_window, center=False).mean() 


    return K, D

In [6]:
df['RSI'] = computeRSI(df['Close'], 14)
df['K'], df['D'] = stochastic(df['RSI'], 3, 3, 14)

In [7]:
df['time UTC'] = pd.to_datetime(df['Open time'], unit='ms', origin='unix')
df.dropna(inplace=True)
df

,Open time,Open,High,Low,Close,RSI,K,D,time UTC
31,1678060831000,22422.12,22423.83,22421.52,22422.54,54.033107,91.624977,95.381293,2023-03-06 00:00:31
32,1678060832000,22422.54,22424.50,22422.31,22422.54,54.033107,88.731052,91.624977,2023-03-06 00:00:32
33,1678060833000,22422.55,22422.73,22422.52,22422.55,54.080566,91.371062,90.575697,2023-03-06 00:00:33
34,1678060834000,22422.53,22423.00,22422.32,22422.32,52.732041,89.725047,89.942387,2023-03-06 00:00:34
35,1678060835000,22422.46,22423.01,22422.16,22422.16,51.765031,86.953478,89.349862,2023-03-06 00:00:35
...,...,...,...,...,...,...,...,...,...
86395,1678147195000,22408.90,22409.75,22408.41,22408.53,50.694848,6.975464,12.929896,2023-03-06 23:59:55
86396,1678147196000,22408.96,22409.84,22408.54,22409.72,58.654142,16.822415,10.257781,2023-03-06 23:59:56
86397,1678147197000,22409.28,22409.28,22408.89,22408.89,52.311174,16.042611,13.280163,2023-03-06 23:59:57
86398,1678147198000,22409.61,22409.73,22408.90,22408.90,52.377994,18.862486,17.242504,2023-03-06 23:59:58


In [8]:
df.loc[(df['D'] <= 0) & (df['RSI'] < 30), 'buy?'] = True
df.loc[df['buy?'] == True]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?
314,1678061114000,22429.06,22429.06,22428.46,22429.03,29.113095,0.0,0.0,2023-03-06 00:05:14,True
496,1678061296000,22427.11,22427.19,22426.48,22426.50,31.933487,0.0,0.0,2023-03-06 00:08:16,True
988,1678061788000,22394.79,22394.97,22391.49,22392.88,14.977936,0.0,0.0,2023-03-06 00:16:28,True
989,1678061789000,22392.53,22393.81,22390.97,22392.52,14.680386,0.0,0.0,2023-03-06 00:16:29,True
990,1678061790000,22392.56,22393.62,22391.62,22392.16,14.372892,0.0,0.0,2023-03-06 00:16:30,True
...,...,...,...,...,...,...,...,...,...,...
84688,1678145488000,22426.99,22427.50,22425.62,22425.67,33.029916,0.0,0.0,2023-03-06 23:31:28,True
85058,1678145858000,22405.84,22406.57,22404.08,22404.66,10.969800,0.0,0.0,2023-03-06 23:37:38,True
85304,1678146104000,22397.52,22397.71,22396.05,22397.00,27.509168,0.0,0.0,2023-03-06 23:41:44,True
85305,1678146105000,22397.00,22397.21,22396.83,22396.83,26.921732,0.0,0.0,2023-03-06 23:41:45,True


In [9]:
rolling_seconds = 60

df['min_val'] = df['Low'].shift(-rolling_seconds).rolling(window=rolling_seconds).min()
df['max_val'] = df['High'].shift(-rolling_seconds).rolling(window=rolling_seconds).max()

df['max_val_%'] = (df['max_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)
df['min_val_%'] = (df['min_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)

#df.dropna(inplace=True)
#df[60:70]
df.loc[df['buy?'] == True]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
314,1678061114000,22429.06,22429.06,22428.46,22429.03,29.113095,0.0,0.0,2023-03-06 00:05:14,True,22418.59,22436.01,0.032369,-0.045299
496,1678061296000,22427.11,22427.19,22426.48,22426.50,31.933487,0.0,0.0,2023-03-06 00:08:16,True,22424.00,22432.00,0.022384,-0.013288
988,1678061788000,22394.79,22394.97,22391.49,22392.88,14.977936,0.0,0.0,2023-03-06 00:16:28,True,22361.43,22395.57,0.013576,-0.138886
989,1678061789000,22392.53,22393.81,22390.97,22392.52,14.680386,0.0,0.0,2023-03-06 00:16:29,True,22361.43,22395.57,0.013442,-0.139019
990,1678061790000,22392.56,22393.62,22391.62,22392.16,14.372892,0.0,0.0,2023-03-06 00:16:30,True,22361.43,22395.57,0.015139,-0.137325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84688,1678145488000,22426.99,22427.50,22425.62,22425.67,33.029916,0.0,0.0,2023-03-06 23:31:28,True,22425.16,22434.69,0.037322,-0.005172
85058,1678145858000,22405.84,22406.57,22404.08,22404.66,10.969800,0.0,0.0,2023-03-06 23:37:38,True,22401.67,22410.49,0.025977,-0.013390
85304,1678146104000,22397.52,22397.71,22396.05,22397.00,27.509168,0.0,0.0,2023-03-06 23:41:44,True,22392.88,22407.52,0.046971,-0.018395
85305,1678146105000,22397.00,22397.21,22396.83,22396.83,26.921732,0.0,0.0,2023-03-06 23:41:45,True,22392.88,22407.52,0.046033,-0.019333


In [10]:
tp_size = 0.075
sl_size = 0.075
# 0.075 RSI 40 total profit 2.47

In [11]:
sl = df.loc[(df['buy?'] == True) & (df['min_val_%'] < -sl_size)]
sl = sl.loc[sl['Open time'] - sl['Open time'].shift(1) > rolling_seconds * 1000]
sl

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
6809,1678067609000,22387.88,22387.88,22385.50,22386.79,27.195847,0.0,0.0,2023-03-06 01:53:29,True,22322.34,22389.12,0.010363,-0.287938
7297,1678068097000,22363.73,22364.27,22362.81,22363.68,20.069246,0.0,0.0,2023-03-06 02:01:37,True,22344.01,22365.33,0.007378,-0.087955
7864,1678068664000,22320.86,22321.49,22319.08,22319.39,33.747747,0.0,0.0,2023-03-06 02:11:04,True,22298.90,22331.70,0.054168,-0.092788
8244,1678069044000,22336.78,22337.11,22336.62,22337.11,39.210785,0.0,0.0,2023-03-06 02:17:24,True,22315.25,22344.40,0.032950,-0.097551
48771,1678109571000,22424.61,22425.49,22423.57,22423.57,35.596605,0.0,0.0,2023-03-06 13:32:51,True,22400.30,22425.74,0.007224,-0.106225
49606,1678110406000,22473.28,22473.34,22468.67,22469.38,34.274563,0.0,0.0,2023-03-06 13:46:46,True,22447.40,22472.06,0.012506,-0.097244
52469,1678113269000,22460.82,22462.03,22459.37,22460.50,34.184385,0.0,0.0,2023-03-06 14:34:29,True,22418.77,22460.53,0.005120,-0.180815
53128,1678113928000,22441.34,22444.96,22438.92,22439.86,35.278416,0.0,0.0,2023-03-06 14:45:28,True,22412.44,22440.83,0.007621,-0.118899
56886,1678117686000,22516.97,22520.08,22516.21,22517.66,35.012457,0.0,0.0,2023-03-06 15:48:06,True,22492.37,22524.32,0.034641,-0.107255
69403,1678130203000,22486.97,22487.19,22485.69,22485.69,22.429764,0.0,0.0,2023-03-06 19:16:43,True,22454.00,22487.97,0.007560,-0.143510


In [12]:
tp = df.loc[(df['buy?'] == True) & (df['max_val_%'] > tp_size)]
tp = tp.loc[tp['Open time'] - tp['Open time'].shift(1) >  rolling_seconds * 1000]
tp

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
4761,1678065561000,22366.61,22367.32,22360.39,22363.84,27.880654,0.0,0.0,2023-03-06 01:19:21,True,22360.01,22382.94,0.089165,-0.013370
7865,1678068665000,22319.61,22319.61,22313.26,22313.26,25.841797,0.0,0.0,2023-03-06 02:11:05,True,22298.90,22331.70,0.082641,-0.064356
57078,1678117878000,22521.11,22522.19,22517.73,22519.31,38.026999,0.0,0.0,2023-03-06 15:51:18,True,22515.10,22599.00,0.354141,-0.018429
58280,1678119080000,22548.01,22548.84,22546.28,22547.02,31.875460,0.0,0.0,2023-03-06 16:11:20,True,22542.72,22563.41,0.075844,-0.015923
59663,1678120463000,22471.90,22472.20,22469.30,22470.91,24.379813,0.0,0.0,2023-03-06 16:34:23,True,22465.60,22488.86,0.079926,-0.023586
69504,1678130304000,22427.96,22430.50,22410.00,22414.28,14.898984,0.0,0.0,2023-03-06 19:18:24,True,22358.00,22438.31,0.097339,-0.260924
69673,1678130473000,22377.19,22377.79,22373.35,22375.32,19.263390,0.0,0.0,2023-03-06 19:21:13,True,22369.00,22407.18,0.141897,-0.028737


In [13]:
tp.shape[0]/sl.shape[0]

0.5833333333333334

In [14]:
(tp.shape[0] * tp_size) - (sl.shape[0] * sl_size)

-0.3749999999999999